# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import hvplot.xarray
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key



In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("/Users/galbeeir/Desktop/git/World_Analysis/World_Weather_Analysis/Starter_Code/starter_code/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,miyaly,48.8844,53.7836,25.14,48,4,3.21,KZ,1688523881
1,1,jamestown,42.0970,-79.2353,21.80,86,0,3.60,US,1688523652
2,2,rawson,-43.3002,-65.1023,9.57,40,0,10.21,AR,1688523882
3,3,bredy,52.4153,60.3411,19.26,81,4,4.79,RU,1688523882
4,4,port-aux-francais,-49.3500,70.2167,4.52,76,100,12.74,TF,1688523882


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [37]:
# Configure the map plot
city_data_df.hvplot.points('Lng', 'Lat', 
                            geo=True,
                            alpha=.50,
                            tiles=True,
                            size='Humidity',
                            frame_height=500,
                            color='City', 
                            cmap='tab10',
                            legend='right',
                            ylim=(-60, 90))


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [98]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[
    (city_data_df['Max Temp'] >=21) & (city_data_df['Max Temp'] <=38) # * Temp
      & (city_data_df['Wind Speed'] < 3) # * Wind Speed
      & (city_data_df['Cloudiness'] == 0) # * Cloudiness 
      ]
# Drop any rows with null values
ideal_weather_df.dropna(how='any')

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
78,78,hamilton,39.1834,-84.5333,26.02,80,0,0.00,US,1688523691
80,80,santa ana de yacuma,-13.7500,-65.4333,23.70,57,0,1.60,BO,1688523903
89,89,bethel,41.3712,-73.4140,23.05,85,0,0.00,US,1688523651
90,90,puerto penasco,31.3167,-113.5333,28.84,82,0,2.24,MX,1688523906
93,93,hurghada,27.2574,33.8129,30.02,25,0,2.06,EG,1688523907
128,128,broome,42.2506,-75.8330,22.79,90,0,1.17,US,1688523919
165,165,cookeville,36.1628,-85.5016,24.45,92,0,0.00,US,1688523930
232,232,newman,37.3138,-121.0208,34.13,43,0,2.24,US,1688523792
245,245,fallon,39.4735,-118.7774,33.93,16,0,2.06,US,1688523953
269,269,metlili chaamba,32.2667,3.6333,31.52,22,0,2.06,DZ,1688523960


### Step 3: Create a new DataFrame called `hotel_df`.

In [99]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.reset_index().loc[:, ['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] =''


# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,hamilton,US,39.1834,-84.5333,80,
1,santa ana de yacuma,BO,-13.7500,-65.4333,57,
2,bethel,US,41.3712,-73.4140,85,
3,puerto penasco,MX,31.3167,-113.5333,82,
4,hurghada,EG,27.2574,33.8129,25,
5,broome,US,42.2506,-75.8330,90,
6,cookeville,US,36.1628,-85.5016,92,
7,newman,US,37.3138,-121.0208,43,
8,fallon,US,39.4735,-118.7774,16,
9,metlili chaamba,DZ,32.2667,3.6333,22,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [100]:
print("Starting hotel search")
# Iterate through the hotel_df DataFrame
for i in range(len(hotel_df)):
    url = "https://api.geoapify.com/v2/places?"
    category = "accommodation.hotel&"
    filter1 = f"filter=circle:{hotel_df['Lng'][i]},{hotel_df['Lat'][i]},5000&"
    bias = f"bias=proximity:{hotel_df['Lng'][i]},{hotel_df['Lat'][i]}&"
    geoapify_url = f"{url}categories={category}{filter1}{bias}apiKey={geoapify_key}"

    # Make and API request using the params dictionaty
    name_address= requests.get(geoapify_url).json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[i, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[i, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[i, 'City']} - nearest hotel: {hotel_df.loc[i, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hamilton - nearest hotel: North Vista Manor
santa ana de yacuma - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
puerto penasco - nearest hotel: Hotel Paraíso Del Desierto
hurghada - nearest hotel: Sea Horse Hotel
broome - nearest hotel: No hotel found
cookeville - nearest hotel: Best Western
newman - nearest hotel: No hotel found
fallon - nearest hotel: Value Inn
metlili chaamba - nearest hotel: No hotel found
maymana - nearest hotel: No hotel found
ypsonas - nearest hotel: Club Aphrodite
paphos - nearest hotel: Mayfair Hotel
kamyzyak - nearest hotel: No hotel found
nassau - nearest hotel: Hilton Garden Inn
argo - nearest hotel: Apollon Hotel
jaen - nearest hotel: Hotel Campos de Baeza
brandon - nearest hotel: Brandon Motor Lodge
ghormach - nearest hotel: No hotel found
moultonborough - nearest hotel: No hotel found
tamanrasset - nearest hotel: فندق الأمان
strelka - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,hamilton,US,39.1834,-84.5333,80,North Vista Manor
1,santa ana de yacuma,BO,-13.7500,-65.4333,57,No hotel found
2,bethel,US,41.3712,-73.4140,85,Hampton Inn Danbury
3,puerto penasco,MX,31.3167,-113.5333,82,Hotel Paraíso Del Desierto
4,hurghada,EG,27.2574,33.8129,25,Sea Horse Hotel
5,broome,US,42.2506,-75.8330,90,No hotel found
6,cookeville,US,36.1628,-85.5016,92,Best Western
7,newman,US,37.3138,-121.0208,43,No hotel found
8,fallon,US,39.4735,-118.7774,16,Value Inn
9,metlili chaamba,DZ,32.2667,3.6333,22,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [102]:
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != 'No hotel found']
hotel_df
# Configure the map plot
hotel_df.hvplot.points('Lng', 'Lat', 
                            geo=True,
                            alpha=.50,
                            tiles=True,
                            size='Humidity',
                            frame_height=500,
                            frame_width=700,
                            color='Hotel Name', 
                            cmap='tab10',
                            legend='right',
                            ylim=(-60, 90),
                            hover_cols=['City', 'Country'])


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity,City,Country)